In [40]:
import re
import json

In [2]:
import cloudscraper
import geopandas as gpd
import h3
import pandas as pd

In [3]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [6]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [7]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

In [8]:
states_df = states_df.replace("United States Virgin Islands", "Virgin Islands")

In [9]:
country_shape_minus_alaska = states_df[states_df["STUSPS"] != "AK"][
    "geometry"
].union_all()

In [24]:
h3_shape = h3.geo_to_h3shape(country_shape_minus_alaska)
h3_cells = h3.h3shape_to_cells(h3_shape, res=4)

## Get Starbucks data

In [25]:
scraper = cloudscraper.create_scraper()

In [54]:
lat, lng = h3.cell_to_latlng("852aa847fffffff")
url = f"https://www.starbucks.com/store-locator?map={lat},{lng}"

In [55]:
locations = scraper.get(url).text

In [56]:
with open("test.txt", "w") as f:
    f.write(locations)

In [57]:
re.findall(r'{"lat":(.*?)\,"lng":(.*?)\}', locations)

[('39.635307', '-101.337891,"zoom":5'),
 ('null',
  'null,"zoomLevel":null,"mapInstance":null,"locatorControlsHeight":0,"locatorPageRectangle":null,"userOverride":false,"preventMovement":false,"coordinatesUpdated":false,"initialUrlZoomOutsideThreshold":false')]

In [18]:
restaurant_list = []
for h3_cell in tqdm(h3_cells, desc="Parsing H3 Cells"):
    lat, lng = h3.cell_to_latlng(h3_cell)
    url = f"https://www.starbucks.com/store-locator?map={lat},{lng}"
    locations = scraper.get(url).text
    restaurants = json.loads(locations)["restaurants"]
    break;

Parsing H3 Cells:   0%|          | 0/30054 [00:00<?, ?it/s]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [19]:
locations = scraper.get(url).text

In [20]:
locations

'<!DOCTYPE html>\n<html lang="en-US" class="no-js fonts-loading">\n\n  <head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta property="x:card" content="summary">\n    <meta property="x:site" content="@starbucks">\n    <meta property="x:creator" content="@starbucksprtnrs">\n    <meta property="og:title" content="Starbucks®">\n    <meta property="og:image" data-hid="image" content="https://app.starbucks.com/weblx/images/social/summary_square.png">\n    <meta property="x:image" data-hid="x:image" content="https://app.starbucks.com/weblx/images/social/summary_image.png">\n    \n    <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no">\n    <script src=\'/vendor/static/vendor2.js\'></script>\n    <script type=\'text/javascript\'>window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"671cca5e0f","applicationID":"24549142","agentToken":null,"a